## Doc2vec implementation, using instead now spaCy

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

model = Doc2Vec(vector_size=50, min_count=2, epochs=40)
tagged_data_train = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(X_train['text_token'])]
data_test = X_train['text_token']
model.build_vocab(tagged_data_train)
model.train(tagged_data_train, total_examples=model.corpus_count, epochs=model.epochs)


def d2v_infer(input):
    vector = model.infer_vector(input)
    return vector.tolist()

docvecs = []
for i in range(0, len(X_train)):
    docvecs.append(list(model.docvecs[i]))

X_train['d2v'] = docvecs
X_train[["d2v" + str(int(x)) for x in np.linspace(start=0, stop=49, num=50)]] = X_train.d2v.apply(pd.Series)
X_train.drop(['text_token', 'd2v'], axis=1, inplace=True)
X_test['d2v'] = X_test['text_token'].progress_apply(d2v_infer)

X_test[["d2v" + str(int(x)) for x in np.linspace(start=0, stop=49, num=50)]] = X_test.d2v.apply(pd.Series)
X_test.drop(['text_token', 'd2v'], axis=1, inplace=True)

# Negation checker lexi method

In [ ]:
nlp = spacy.load('de_core_news_lg')
neg_words = ['nicht', 'kein', 'nirgends', 'nirgendwo', 'niemand', 'niemals', 'nirgendwohin', 'nie', 'nix']

In [ ]:
def senti_class_full(input):
    doc = nlp(input)
    tmp_list = []
    for sent in doc.sents:
        for token in sent:
            lemma = str(tagger.analyze(str(token))[0].lower())
            if lemma in neg_words:
                continue
            children = [tagger.analyze(child.text)[0].lower() for child in token.children]
            negation = 1
            if any(neg_word in children for neg_word in neg_words):
                negation = -1
            if lemma in senti_pos_dict:
                tmp_list.append(senti_pos_dict[lemma]*senti_weight_dict[lemma]*negation)
            if lemma in senti_neg_dict:
                tmp_list.append(senti_neg_dict[lemma]*senti_weight_dict[lemma]*negation)
    return sum(tmp_list)

# Old BERT

In [ ]:
class SentimentModel():
    def __init__(self, model_name: str):
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'

        self.model = AutoModelForSequenceClassification.from_pretrained(model_name).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.clean_chars = re.compile(r'[^A-Za-züöäÖÜÄß ]', re.MULTILINE)
        self.clean_http_urls = re.compile(r'https*\\S+', re.MULTILINE)
        self.clean_at_mentions = re.compile(r'@\\S+', re.MULTILINE)

    def predict_sentiment(self, texts):
        if np.any(pd.isna(texts)):
          return np.nan
        texts = [self.clean_text(str(text)) for text in [texts]]
        # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
        encoded = self.tokenizer.batch_encode_plus(texts,padding=True, add_special_tokens=True,truncation=True, return_tensors="pt")
        encoded = encoded.to(self.device)
        with torch.no_grad():
                logits = self.model(**encoded)

        label_ids = torch.argmax(logits[0], axis=1)
        for a in zip(logits[0].tolist(), [self.model.config.id2label[label_id] for label_id in label_ids.tolist()]):
            tmp_rsl = list(a)
        return tmp_rsl

    def replace_numbers(self,text: str) -> str:
            return text.replace("0"," null").replace("1"," eins").replace("2"," zwei").replace("3"," drei").replace("4"," vier").replace("5"," fünf").replace("6"," sechs").replace("7"," sieben").replace("8"," acht").replace("9"," neun")

    def clean_text(self,text: str)-> str:
            text = text.replace("\n", " ")
            text = self.clean_http_urls.sub('',text)
            text = self.clean_at_mentions.sub('',text)
            text = self.replace_numbers(text)
            text = self.clean_chars.sub('', text) # use only text chars
            text = ' '.join(text.split()) # substitute multiple whitespace with single whitespace
            text = text.strip().lower()
            return text